In [1]:
library(tidyverse)
library(testthat)
library(digest)
library(repr)
library(tidymodels)
library(cowplot)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘testthat’


The following object is masked from ‘package:dplyr’:

    matches


The following object is masked from ‘package:purrr’:

    is_null


The following object is masked from ‘package:tidyr’:

    matches


Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────

In [12]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv"
pollution <- read_csv(url)
pollution
pollution[is.na(pollution)] <- 0
head(pollution)

Parsed with column specification:
cols(
  No = col_double(),
  year = col_double(),
  month = col_double(),
  day = col_double(),
  hour = col_double(),
  pm2.5 = col_double(),
  DEWP = col_double(),
  TEMP = col_double(),
  PRES = col_double(),
  cbwd = col_character(),
  Iws = col_double(),
  Is = col_double(),
  Ir = col_double()
)



No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,2010,1,1,0,NA,-21,-11,1021,NW,1.79,0,0
2,2010,1,1,1,NA,-21,-12,1020,NW,4.92,0,0
3,2010,1,1,2,NA,-21,-11,1019,NW,6.71,0,0
4,2010,1,1,3,NA,-21,-14,1019,NW,9.84,0,0
5,2010,1,1,4,NA,-20,-12,1018,NW,12.97,0,0
6,2010,1,1,5,NA,-19,-10,1017,NW,16.10,0,0
7,2010,1,1,6,NA,-19,-9,1017,NW,19.23,0,0
8,2010,1,1,7,NA,-19,-9,1017,NW,21.02,0,0
9,2010,1,1,8,NA,-19,-9,1017,NW,24.15,0,0


No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,2010,1,1,0,0,-21,-11,1021,NW,1.79,0,0
2,2010,1,1,1,0,-21,-12,1020,NW,4.92,0,0
3,2010,1,1,2,0,-21,-11,1019,NW,6.71,0,0
4,2010,1,1,3,0,-21,-14,1019,NW,9.84,0,0
5,2010,1,1,4,0,-20,-12,1018,NW,12.97,0,0
6,2010,1,1,5,0,-19,-10,1017,NW,16.10,0,0


In [3]:
set.seed(9999)

In [13]:
pollution_split <- initial_split(pollution, prop = 3/4, strata = pm2.5)
pollution_training <- training(pollution_split)
pollution_testing <- testing(pollution_split)

pollution_split
pollution_training
pollution_testing

<Analysis/Assess/Total>
<32869/10955/43824>

No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,2010,1,1,0,0,-21,-11,1021,NW,1.79,0,0
2,2010,1,1,1,0,-21,-12,1020,NW,4.92,0,0
3,2010,1,1,2,0,-21,-11,1019,NW,6.71,0,0
4,2010,1,1,3,0,-21,-14,1019,NW,9.84,0,0
5,2010,1,1,4,0,-20,-12,1018,NW,12.97,0,0
6,2010,1,1,5,0,-19,-10,1017,NW,16.10,0,0
8,2010,1,1,7,0,-19,-9,1017,NW,21.02,0,0
10,2010,1,1,9,0,-20,-8,1017,NW,27.28,0,0
11,2010,1,1,10,0,-19,-7,1017,NW,31.30,0,0


No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
7,2010,1,1,6,0,-19,-9,1017,NW,19.23,0,0
9,2010,1,1,8,0,-19,-9,1017,NW,24.15,0,0
15,2010,1,1,14,0,-18,-2,1014,NW,43.82,0,0
16,2010,1,1,15,0,-18,-1,1014,cv,0.89,0,0
17,2010,1,1,16,0,-19,-2,1015,NW,1.79,0,0
18,2010,1,1,17,0,-18,-3,1015,NW,2.68,0,0
19,2010,1,1,18,0,-18,-5,1016,NE,1.79,0,0
21,2010,1,1,20,0,-17,-5,1017,cv,0.89,0,0
32,2010,1,2,7,124,-7,-5,1024,SE,10.72,0,0
